# 🛠 Step 02: 개발 환경 구성

이 노트북에서는 RAG 시스템 개발에 필요한 환경을 구성합니다.

## 📌 필요한 패키지 설치

먼저 필요한 패키지들을 설치하겠습니다:

In [ ]:
# RAG 시스템 구축에 필요한 주요 패키지들을 설치
# langchain: LLM 애플리케이션 개발을 위한 프레임워크
# openai: OpenAI API를 사용하기 위한 공식 클라이언트
# elasticsearch: 벡터 검색을 위한 Elasticsearch 클라이언트
# python-dotenv: 환경 변수 관리를 위한 패키지
# pandas, numpy: 데이터 처리를 위한 기본 패키지들


!pip install langchain==0.1.0 \
            openai==1.12.0 \
            elasticsearch==8.11.1 \
            python-dotenv==1.0.0 \
            pandas==2.1.4 \
            numpy==1.26.3

## 🔑 환경 변수 설정

프로젝트 루트에 `.env` 파일을 생성하고 필요한 환경 변수를 설정합니다:

In [ ]:
# %%writefile 매직 커맨드를 사용하여 프로젝트 루트에 .env 파일 생성
# OPENAI_API_KEY: OpenAI API 인증을 위한 키
# ELASTICSEARCH_URL: Elasticsearch 서버 접속 URL (k8s 서비스 주소)
# ELASTICSEARCH_USERNAME: Elasticsearch 접속 계정
# ELASTICSEARCH_PASSWORD: Elasticsearch 접속 비밀번호

%%writefile ../.env
OPENAI_API_KEY=your-api-key-here
ELASTICSEARCH_URL=https://elasticsearch-master.elasticsearch.svc.cluster.local:9200
ELASTICSEARCH_USERNAME=elastic
ELASTICSEARCH_PASSWORD=your-password-here

## 🔍 Elasticsearch 연결 테스트

Elasticsearch가 정상적으로 동작하는지 확인합니다:

In [ ]:
# 필요한 라이브러리 임포트
import os  # 환경 변수 접근을 위한 모듈
from dotenv import load_dotenv  # .env 파일에서 환경 변수를 로드하기 위한 모듈
from elasticsearch import Elasticsearch  # Elasticsearch 클라이언트

# .env 파일에서 환경 변수를 시스템에 로드
load_dotenv()

# Elasticsearch 클라이언트 인스턴스 생성
es = Elasticsearch(
    os.getenv('ELASTICSEARCH_URL'),  # .env 파일에서 설정한 Elasticsearch 서버 주소
    basic_auth=(  # 기본 인증 정보 설정
        os.getenv('ELASTICSEARCH_USERNAME'),  # Elasticsearch 사용자 이름
        os.getenv('ELASTICSEARCH_PASSWORD')   # Elasticsearch 비밀번호
    ),
    verify_certs=False  # SSL 인증서 검증 비활성화 (개발 환경에서만 사용할 것)
)

# Elasticsearch 서버 연결 상태 확인
if es.ping():  # ping() 메서드로 서버 응답 확인
    print("✅ Elasticsearch 연결 성공!")
    print(f"📊 클러스터 정보: {es.info()}")  # 클러스터 상세 정보 출력
else:
    print("❌ Elasticsearch 연결 실패")

## 🔄 OpenAI API 테스트

OpenAI API가 정상적으로 작동하는지 확인합니다:

In [ ]:
# OpenAI API 클라이언트 라이브러리 임포트
from openai import OpenAI

# OpenAI 클라이언트 인스턴스 생성 (자동으로 OPENAI_API_KEY 환경 변수 사용)
client = OpenAI()

# OpenAI API 연결 테스트
try:
    # text-embedding-ada-002 모델을 사용하여 텍스트 임베딩 생성
    response = client.embeddings.create(
        model="text-embedding-ada-002",  # OpenAI의 텍스트 임베딩 모델
        input="Hello, World!"  # 테스트용 입력 텍스트
    )
    print("✅ OpenAI API 연결 성공!")
    print(f"📊 임베딩 차원: {len(response.data[0].embedding)}")  # 생성된 임베딩의 차원 수 출력
except Exception as e:
    print(f"❌ OpenAI API 연결 실패: {e}")  # 오류 발생 시 예외 메시지 출력

## 🎯 다음 단계

개발 환경 설정이 완료되었습니다. 다음 단계에서는 Iceberg metadata를 로딩하고 분석하겠습니다.